In [1]:
from snowflake.snowpark import Session

from tauntaun.agents import Analyst, DictMessageStore

session = Session.builder.config("connection_name", "<YOUR_CONNECTION_NAME>").create()

messages = DictMessageStore()

analyst_1 = Analyst(
    session=session,
    message_store=messages,
    semantic_model_full_path="cortex_analyst_demo.revenue_timeseries.raw_data/revenue_timeseries.yaml",
)

In [2]:
analyst_1.process_message(prompt="What is the revenue for the year 2024?")
print("State:\n", analyst_1.get_state(), "\n")
print("Results:\n", analyst_1.get_results(), "\n")

State:
 {'text': 'This is our interpretation of your question:\n\n__What is the total revenue for the year 2024?__ \n\n', 'sql': "WITH __daily_revenue AS (\n  SELECT\n    date,\n    revenue AS daily_revenue\n  FROM cortex_analyst_demo.revenue_timeseries.daily_revenue\n)\nSELECT\n  SUM(dr.daily_revenue) AS revenue_2024\nFROM __daily_revenue AS dr\nWHERE\n  DATE_PART('YEAR', dr.date) = 2024\n -- Generated by Cortex Analyst\n;"} 

Results:
    REVENUE_2024
0      966657.0 



In [3]:
analyst_1.process_message(prompt="can you break it out by region?")
print("State:\n", analyst_1.get_state(), "\n")
print("Results:\n", analyst_1.get_results(), "\n")

State:
 {'text': 'This is our interpretation of your question:\n\n__What is the total revenue breakdown by sales region for the year 2024?__ \n\n', 'sql': "WITH __daily_revenue AS (\n  SELECT\n    region_id,\n    date,\n    revenue AS daily_revenue\n  FROM cortex_analyst_demo.revenue_timeseries.daily_revenue\n), __region AS (\n  SELECT\n    region_id,\n    sales_region\n  FROM cortex_analyst_demo.revenue_timeseries.region_dim\n)\nSELECT\n  r.sales_region,\n  SUM(d.daily_revenue) AS total_revenue\nFROM __daily_revenue AS d\nLEFT OUTER JOIN __region AS r\n  ON d.region_id = r.region_id\nWHERE\n  DATE_PART('YEAR', d.date) = 2024\nGROUP BY\n  r.sales_region\nORDER BY\n  total_revenue DESC NULLS LAST\n -- Generated by Cortex Analyst\n;"} 

Results:
     SALES_REGION  TOTAL_REVENUE
0           Asia     1033358.45
1  South America     1013763.35
2         Africa     1006185.15
3  North America      890453.55
4         Europe      889524.50 

